### 1. Нужно дедуктивно догадаться и написать аннотации типов к функции calc_summary_free_space

In [62]:
import typing as tp

T = tp.TypeVar('T', int, float)  # Есть абстракции Numbers, но вроде как Mypy не умеет работать с этой абстракцией

def calc_summary_free_space(bucket_space_info: tp.Mapping[tp.Any, T]) -> T:
    if not bucket_space_info:
        return None
    total_free_space = 0
    for bucket_name, bucket_free_space in bucket_space_info.items(): # Здесь ответ
        if bucket_name in ('dev_team', 'users', 'tech_support'):
            total_free_space += bucket_free_space
    return total_free_space


calc_summary_free_space({'dev_team': 21.0, 'users': 5, 'abc': 55, 5:5})

26.0

Решение в голове цикла. Мы получаем итерируемый объект, запрашивая ключ и значение (словерю-подобный объект). Не обязательно словарь, главное чтобы к объекту можно было обращаться по ключу (Mapping).

Ключ может быть любого типа, код как не поламается, а вот значение обязательно число (вроде как можно float - т.к. класс более обширен, чем int и bool.

Возвращаем число.

#########################

**Задача 2**

Есть код, который формирует список объектов автомобилей.

Задачи:
1. Стандартными механизмами отсортировать список одновременно по двум полям: пробег, цена.
2. При помощи списковых включений отфильтровать список так: все автомобили с двигателем больше 1.4 кроме черных.
3. Повторить пункт 2, но вместо списковых включений использовать функцию filter.


In [63]:
from typing import List


class Car:
    def __init__(self, year: int, distance: int, color: str, cost: int, engine: float):
        self.year: int = year
        self.distance: int = distance
        self.color: str = color
        self.cost: int = cost
        self.engine: float = engine

    def __repr__(self):
        return f'Car({self.color}, {self.distance}, {self.year}, {self.engine}, {self.cost})'


def build_cars_objects() -> List[Car]:
    dataset = [
        (2014, 80_000, 'Black', 450_000, 1.4),
        (2019, 0, 'Silver', 1_100_000, 1.6),
        (2018, 20_000, 'Blue', 700_000, 1.4),
        (2016, 120_000, 'White', 505_000, 2.0),
        (2014, 185_000, 'Red', 330_000, 1.4),
        (2015, 35_000, 'Black', 890_000, 2.0),
        (1998, 240_000, 'Silver', 15_000, 1.3),
        (2020, 0, 'White', 2_500_000, 3.0),
        (2018, 65_000, 'Yellow', 445_000, 1.6),
        (2009, 100_000, 'Red', 250_000, 1.4),
        (2021, 0, 'Black', 1_499_000, 1.8),
    ]

    cars = [Car(*fields) for fields in dataset]
    return cars


cars = build_cars_objects()

In [64]:
# 1. Стандартными механизмами отсортировать список одновременно по двум полям: пробег, цена.
sorted(cars, key=lambda car: (car.distance, car.cost)) # Если в другом порядке, то reverse=True

[Car(Silver, 0, 2019, 1.6, 1100000),
 Car(Black, 0, 2021, 1.8, 1499000),
 Car(White, 0, 2020, 3.0, 2500000),
 Car(Blue, 20000, 2018, 1.4, 700000),
 Car(Black, 35000, 2015, 2.0, 890000),
 Car(Yellow, 65000, 2018, 1.6, 445000),
 Car(Black, 80000, 2014, 1.4, 450000),
 Car(Red, 100000, 2009, 1.4, 250000),
 Car(White, 120000, 2016, 2.0, 505000),
 Car(Red, 185000, 2014, 1.4, 330000),
 Car(Silver, 240000, 1998, 1.3, 15000)]

In [65]:
# 2. При помощи списковых включений отфильтровать список так: все автомобили с двигателем больше 1.4 кроме черных.
[car for car in cars if car.engine > 1.4 and car.color != 'Black']

[Car(Silver, 0, 2019, 1.6, 1100000),
 Car(White, 120000, 2016, 2.0, 505000),
 Car(White, 0, 2020, 3.0, 2500000),
 Car(Yellow, 65000, 2018, 1.6, 445000)]

In [66]:
# 3. Повторить пункт 2, но вместо списковых включений использовать функцию filter.
list(filter(lambda car: car.engine > 1.4 and car.color != 'Black', cars))

[Car(Silver, 0, 2019, 1.6, 1100000),
 Car(White, 120000, 2016, 2.0, 505000),
 Car(White, 0, 2020, 3.0, 2500000),
 Car(Yellow, 65000, 2018, 1.6, 445000)]

#########################

**Задача 3**

Эта функция поиска всех питоновских файлов в каталоге, начинающихся не с точки и не с подчеркивания.
В функции ошибка, которая приводит к переполнению памяти OS и падению сервера.

Нужно догадаться, что может приводить к ошибке.

In [67]:
import os


def get_python_files(path):
    files = [] # не локальная переменная в цикле ниже!
    fs_objects = os.walk(path)
    
    for folder, subfolder, files in fs_objects:
        if folder.startswith('.'):
            continue
        if folder.startswith('_'):
            continue
        # Здесь files локальная переменная полученная из fs_objects. Бесконечное пополнение списка файлами.
        # В цикле список постоянно пополняется и мы уходим в бесконечный цикл, забивая всю память
        for file in files: 
            if file.startswith('_'):
                continue
            if not file.endswith('.py'):
                continue
            files.append(f'{folder}/{file}')  # file добавлен в files, лист пополнен=)
    return files

Указал в комментариях к коду.

Суть простая, пустой список files - для заполнения в начале кода, заменяется локальной ссылкой в цикле итератора fs_objects.

Соответственно, в цикле "file in files" мы после каждого прохода удовлетворяющего условиям, добавляем в конец локальной переменной новый file. Список никогда не заканчивается, уходим в бесконечный цикл, пока не скушаем всю память.

Ниже рабочее решение:

In [68]:
def get_python_files(path):
    file_list = []  # уникальное имя
    fs_objects = os.walk(path)

    for folder, subfolder, files in fs_objects:
        if folder.startswith('.'):
            continue
        if folder.startswith('_'):
            continue
        for file in files:
            if file.startswith('_'):
                continue
            if not file.endswith('.py'):
                continue
            file_list.append(f'{folder}/{file}')  # В локальной области цикла ссылки нет, ищем во внешней.
    return file_list

#######################

**Задача 4**

Есть функция `expansive_api()`, которая делает платный (дорогой) API-вызов.
Нужно, чтобы функция помнила, сколько раз её вызвали, и чтобы на третий раз она выбросила ошибку:
`TooExpensive('Too expansive call')`.
При этом вызов функции должен остаться неизменным (неизменяемая секция ниже не должна сломаться)

Нужно перечислить возможные варианты, которые приходят в голову, как это сделать (есть как минимум 3) и один из них реализовать.

In [69]:
def expansive_api():
    print('performing expansive call')

# --- неизменяемая секция ---
expansive_api()
expansive_api()
expansive_api()

performing expansive call
performing expansive call
performing expansive call


1. Декоратор. Оборачиваем функцию, если счетчик вызовов равен 0 - вызываем и локально сохраняем результат, если 1 - возвращаем результат, 2 - ошибка и сброс счетчика. Ну можно еще контроль времени, но это усложняет логику и вне ТЗ). Думаю, это лучший способ.
2. Можем запилить подобный счетчик и логику, но в теле самой функции (имхо, худший вариант с т.з. архитектуры). Можем присвоить функции локальную переменную, типа expansive_api.cnt = 0 и уже от нее писать логику (при вызове функции увеличивать). Придется прибегнуть к замыканиям.
3. Можем написать производную от данной функции, хранящую счетчик ссылок и дать ей то же название. По мне, те же яйца, что и в п.1, только в профиль

In [70]:
# Реализую первый

class TooExpensive(Exception):
    
    def __init__(self, text_error=None):
        self.text_error = text_error
        
    def __str__(self):
        return self.text_error


class CallDecorator:
    
    def __init__(self, max_call):
        self.max_call = max_call  # Вызов, вызывающий исключение
        self.cnt = 0  # Счетчик
        self.result = self.func = None  # Результат и функции пока не определены

        
    def __call__(self, func=None): # Работает только для функций без условий (что логично=) )
        if func is not None:
            self.func = func
            return self
        
        self.cnt += 1
        if self.cnt == 1:
            self.result = self.func()
        elif self.cnt >= self.max_call:
            raise TooExpensive('Too expansive call')
        return self.result
        

In [74]:
@CallDecorator(3) # На 3й выкидываем
def expansive_api():
    return 'performing expansive call' # Функция с принтом ничего не возвращает

# --- неизменяемая секция ---
print(expansive_api())
print(expansive_api())
print(expansive_api())

performing expansive call
performing expansive call


TooExpensive: Too expansive call

#####################

**Задача 5**

Есть некий исполнитель `Performer` абстрактных задач. И есть две разные задачи `CreateEntry` и `DeleteEntry`.
1. Нужно сделать утиную типизацию таким образом, чтобы исполнитель умел выполнять любую подходящую задачу.
2. Нужно при помощи инверсии управления (IoC) сделать возможным в самой задаче переопределять поведение инициализации и завершения задачи, а так же реакцию на ошибку.
3. Для демонстрации на основе этой архитектуры определить в `CreateEntry` поведение для подключения/отключения БД перед и после выполнения задачи. А в `DeleteEntry` определить поведение в случае ошибки. В качестве стабов можно использовать принты типа `print('connect to DB')`

С IoC не работал. Но если я понял правильно, код ниже:

In [75]:
class Performer:
    creater: 'CreateEntry' = None
    deleter: 'DeleteEntry' = None
    
    def __new__(cls):
        if not hasattr(cls, 'instance'):
            cls.instance = super().__new__(cls)
            cls.creater = CreateEntry()
            cls.deleter = DeleteEntry()
        return cls.instance
    
    def work1(self):
        print('work1')
        
    def work2(self):
        print('work2')
        
        
    def perform(self, task: str):  # Будем предпологать, что все задачи без условий
        self.creater.connect()
        result = None
        try:
            result = self.__getattribute__(task)()
        except:
            self.deleter.do_if_error()
        finally:
            self.creater.disconect()
        return result
    

class CreateEntry:
    
    def connect(self):
        print('connect to DB')
        
        
    def disconect(self):
        print('disconect DB')


class DeleteEntry:
    def do_if_error(self):
        print('\n\nWORK WITH ERROR \n\n ')
        

In [76]:
perf = Performer()
perf.perform('work1')
print('-'*20)
perf.perform('work2')
print('-'*20)
perf.perform('work3')

connect to DB
work1
disconect DB
--------------------
connect to DB
work2
disconect DB
--------------------
connect to DB


WORK WITH ERROR 

 
disconect DB


################################

**Задача 6**

Ниже приведен код. Нужно объяснить, что там происходит и зачем.

In [ ]:
import unittest
from unittest.mock import patch


def get_data_from_file(): # Метод возвращает some,data
    print('Reading from file')
    return 'some,data'


def format_data():
    data = get_data_from_file()
    return data.replace(',', ', ').capitalize() #some,data --> Some, data, замена и первая буква с заглавной


class Test1(unittest.TestCase):
    def test_format_data(self):
        with patch('__main__.get_data_from_file') as mock_get_data_from_file: # Получаем объект для подмены (mock)
             # Теперь наша функция при вызове возвращает 'hello,world'
            mock_get_data_from_file.return_value = 'hello,world'
            result = format_data()  # из этой функции вызывается get_data_from_file (mock)
            assert result == 'Hello, world' # Отработан верно
            mock_get_data_from_file.assert_called_once() # Проверка, что mock вызывался не более одного раза


unittest.main()  # старт тестов!

По тесту
1. Получаем объект для подмены (mock)
2. Теперь наша функция (get_data_from_file) при вызове возвращает 'hello,world'
3. Из функции format_data вызывается get_data_from_file (mock)
4. Проверяем результат
5. Проверяем, что mock вызывался более одного раза

Примечание - код отработал в Pycharm, но указать пришлось не `__main__`, a имя модуля

######################################

Задача 7

Есть кривой асинхронный код, который на самом деле синхронный. Нужно его исправить, чтоб он стал асинхронным.

In [ ]:
import asyncio


async def connect_db_1():
    print('Start connecting to DB_1')
    await asyncio.sleep(5)  # slow connect to DB_1
    print('Finish connecting to DB_1')


async def connect_db_2():
    print('Start connecting to DB_2')
    await asyncio.sleep(3)  # slow connect to DB_2
    print('Finish connecting to DB_2')


async def app_startup():
    await connect_db_1() # Запускаем сопрограмму, ждем результата. Формально, это блок основного потока
    await connect_db_2() # Так как этот вызов находидтся в одном потоке, он не запустится, пока не отработает connect_db_1

asyncio.run(app_startup())

In [ ]:
# Так заработает
async def app_startup():
    # Создаем две таски, запускаемые в цикле событий
    task1 = asyncio.create_task(connect_db_1())
    task2 = asyncio.create_task(connect_db_2())
    # Теперь работа асинхронна 
    await task1
    await task2

Примечание: при таком подходе task2 все же зависим от task1 (возврат результата). Но выполняются они асинхронно

#######################################

---

**Задача 8**

Есть скрипт получения пользователя из БД, стэк вызовов такой:

```
main_entrypoint(user_id)
╰-> get_user(user_id)
    ╰-> get_user_data(user_id)
        ╰-> get_data_from_db(query)
            ╰-> db_is_available()
```


Если БД недоступна, то через весь стэк до верхнего уровня передается `None`,
обременяя каждую функцию лишними обязанностями проверки на `None`.
Кроме того, это усложняет типизацию, мы почти везде вынуждены добавлять `Optional`.

Как решить эти две проблемы?

In [ ]:
import random
import logging
from typing import Dict, Optional

logger = logging.getLogger(__name__)


# level 5 Провека доступа
def db_is_available() -> bool:
    return random.choice([True, False])  # имитация случайного отключения БД


# level 4 Select
def get_data_from_db(query) -> Optional[Dict]:
    if not db_is_available():
        return None
    else:
        return {'request': 'result'}


# level 3 формирование запроса
def get_user_data(user_id: int) -> Optional[Dict]:
    query = f'SELECT * FROM users WHERE id={user_id}'
    user_data = get_data_from_db(query)
    if user_data is None:
        return None
    return user_data


# level 2 получение конкретных данных
def get_user(user_id: int) -> Optional[Dict]:
    user_data = get_user_data(user_id)
    if user_data is None:
        return None
    user = {'user': 'data'}
    return user

# level 1 логирование
def main_entrypoint(user_id: int) -> Optional[Dict]:
    user = get_user(user_id)
    if user is None:
        logger.error(f'Failed to sync user {user_id}')
        return None
    else:
        logger.info(f'Sync user {user_id} success')
        return user

Проверку доступности нужно провести в первую очередь, сразу убрав вызов функций в стеке. В остальных случаях мы всегда получим dict. Но как я понял, если нет юзера в БД, то получим пустой объект {'request': None}.

In [77]:
import random
import logging
from typing import Dict, Optional

logger = logging.getLogger(__name__)


# level 5 Провека доступа
def db_is_available() -> bool:
    return random.choice([True, False])  # имитация случайного отключения БД


# level 4 Select
def get_data_from_db(query) -> Dict:
    result = {'request': 'result'}
    return result


# level 3 формирование запроса
def get_user_data(user_id: int) -> Dict:
    query = f'SELECT * FROM users WHERE id={user_id}'
    return get_data_from_db(query)

# level 2 получение конкретных данных
def get_user(user_id: int) -> Dict:
    user_data = get_user_data(user_id)
    user = {'user': 'data'}
    return user

# level 1 логирование
def main_entrypoint(user_id: int) -> Optional[Dict]:
    if db_is_available():
        user = get_user(user_id)
        logger.info(f'Sync user {user_id} success')
        return user 
    logger.error(f'Failed to sync user {user_id}')

#####################################################

**Задача 9**

Что делает эта строка?

`grep -E '(sudo|ssh)' /var/log/auth.log`


**Линух не админил, простите=)**

---

**Задача 10**

Есть многопоточная программа, которая куда-то пишет данные (в консоль или файл).
Ниже приведен кусок этого вывода, где строки наслаиваются друг на друга.
Какой ситуацией может быть вызван такой эффект и как это исправить?


```
2022-02-02 09:09:22.930163 Some sample data
2022-02-02 09:20:18.569518 Some sample data
2022-02-02 10:48:52.062795 Some sample data
2022-02-02 18:25:03.978288 Some sample data
2022-02-03 12:212022-02-03 14:18:33.833549 Some sample data
:22.955283 Some sample data
2022-02-05 08:04:41.224744 Some sample data
2022-02-06 08:10:15.876875 Some sample data
2022-02-15 10:58:29.634397 Some sample data
2022-02-15 11:00:41.260224 Some sample data
2022-02-15 11:03:05.161016 Some sample data
2022-02-15 11:05:25.318107 Some sample data
```


Проблема в том, что к консоли или файлу одновременно обращаются несколько потоков. Вначале пишет один, затем GIL перехватывает другой поток и начинает писать свои данные. Мы и видим эту проблему.
Обходится это механизмом блокировок=)

#################################################

**Задача 11**

Нужно придумать схему хранения пользователей их постов и лайков к постам. У каждого пользователя есть логин, имя и телефон. У поста есть заголовок, текст и дата поста.

Нужно устно продумать схему таблиц, отношений, перечислить возможные поля, примерные типы данных, индексы, ключи. Чем больше всего будет продумано, тем лучше.

Картинка с БД будет приложена в github.

**USERS**

    PK	 id	INTEGER PK IDENTITY
        login	VARCHAR(100) NOT NULL UNIQUE
        name	 NVARCHAR(100) NOT NULL
        phone	BIGINT UNIQUE
        
**POSTS**

    PK	post_id	BIGINT PK IDENTITY
    FK	user_id	INTEGER NOT NULL
          title	  NVARCHAR(300) NOT NULL
          text	   TEXT NOT NULL
          date	   DATETIME NOT NULL DEFAULT GETDATETIME()


**LIKES**

        FK	post_id	BIGINT
        FK	user_id	INTEGER


Три таблицы с полями указанными выше (плюс тип поля, уникальное значение, первичный/внешний ключ, дефолтное значение).
Индексировать не вижу смысла - FK ссылки, PK индексируется по умолчанию. Дату можно определять на строне СУБД (плюс ID идет по возрастанию, т.е. его нет смыслк как то индексировать). Связи c таблицами по FK. LIKES - многие ко многим. При удалении записей установить каскадное удаление (пропал юзер, пропали его посты и лайки по PK-FK)

Понятно, что это ограниченное решение (нет, например, таблиц с корректировками и прочим, но +/- пойдет, имхо)